# Driver State Anomaly Detection With Temporal Autoencoders

[https://dagshub.com/matejfric/driver-state](https://dagshub.com/matejfric/driver-state)

TODO: setup a pre-commit hook for https://github.com/mwouts/jupytext and Ruff

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime
import os
import re
from collections import defaultdict
from collections.abc import Mapping
from itertools import chain
from pathlib import Path
from typing import Literal

import dagshub
import matplotlib.pyplot as plt
import mlflow
import mlflow.pytorch
import pytorch_lightning as L
import torch
import torch.nn.functional as F
import torchview
from mlflow.models.signature import infer_signature
from pytorch_lightning.callbacks import TQDMProgressBar

# Pytorch Lightning EarlyStopping callback does not recover the best weights as in Keras!
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

# https://github.com/Lightning-AI/pytorch-lightning/discussions/10399,
# https://pytorch-lightning.readthedocs.io/en/1.5.10/extensions/generated/pytorch_lightning.callbacks.ModelCheckpoint.html
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger
from sklearn.metrics import roc_auc_score
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

from model.ae import (
    LSTMDecoder,
    LSTMEncoder,
    TemporalAutoencoderModel,
    summarize_model,
)
from model.ae.temporal_3d import (
    Conv3dDecoder,
    Conv3dEncoder,
)
from model.common import Anomalies, BatchSizeDict
from model.dataset import TemporalAutoencoderDataset
from model.git import get_commit_id, get_current_branch
from model.plot import (
    plot_learning_curves,
    plot_roc_chart,
    plot_temporal_autoencoder_reconstruction,
    show_examples,  # noqa F401 TODO
    show_random,  # noqa F401 TODO
)

In [ ]:
# from torch import nn

# rnn = nn.LSTM(128, 20, 1, batch_first=True, bidirectional=True)
# input = torch.randn(32, 4, 128)
# output, (hn, cn) = rnn(input)
# print(output.shape)
# print(hn.shape)
# print(cn.shape)

## Configuration

In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'font.size': 14})

# Experiment logging
REPO_NAME = 'driver-state'
USER_NAME = 'matejfric'
dagshub.init(REPO_NAME, USER_NAME, mlflow=True)  # type: ignore

# Reproducibility
# https://lightning.ai/docs/pytorch/stable/common/trainer.html#reproducibility
SEED = 42
L.seed_everything(SEED, workers=True)

print(
    f'torch: {torch.__version__}, cuda: {torch.cuda.is_available()}, lightning: {L.__version__}'  # type: ignore
)

In [ ]:
# HYPERPARAMETERS
# ----------------------------------------
MAX_EPOCHS = 10
MONITOR = 'valid_loss'
PATIENCE = 3
# Run memory map script to use different image size (`run_memory_map_conversion.py`)
IMAGE_SIZE: Literal[128, 224, 256] = 128
BATCH_SIZE = 32
TIME_STEPS = 2
LEARNING_RATE = 0.0005  # 1e-4
LOSS_FUNCTION = 'mse'  # 'mae'
TRAIN_NOISE_STD_INPUT = 0.0  # 0.025
TRAIN_NOISE_STD_LATENT = 0.0  # 0.025
TRAIN_SET_RATIO = 0.9

# LOGGING
# ----------------------------------------
MODEL_NAME: Literal['tae', 'tae3d'] = 'tae'
LOG_DIR = Path('logs')
EXPERIMENT_NAME = f'{datetime.datetime.now().strftime("%Y-%m-%d-%H%M%S")}-{MODEL_NAME}'
VERSION = 0
EXPERIMENT_DIR = LOG_DIR / EXPERIMENT_NAME / f'version_{VERSION}'
DATASET_NAME = '2024-10-28-driver-all-frames/2021_08_31_geordi_enyaq'

MLFLOW_ARTIFACT_DIR = 'outputs'
METRICS_CSV_NAME = 'metrics.csv'
LEARNING_CURVES_PDF_NAME = 'learning_curves.pdf'
PREDICTIONS_PNG_NAME = 'predictions.png'
TRAIN_TRANSFORMS_JSON_NAME = 'train_transforms.json'
NOTEBOOK_NAME = 'temporal_anomaly_detection.ipynb'
ARCHITECTURE_VISUALIZATION_NAME = 'architecture'
ROC_CHART_NAME = 'roc_chart.png'

# DATASET
# ----------------------------------------
DATASET_DIR = Path().home() / f'source/driver-dataset/{DATASET_NAME}'

NORMAL_MEMORY_MAP = DATASET_DIR / 'normal' / 'memory_maps' / f'depth_{IMAGE_SIZE}.dat'
ANOMAL_MEMORY_MAP = DATASET_DIR / 'anomal' / 'memory_maps' / f'depth_{IMAGE_SIZE}.dat'
ANOMALIES_FILE = DATASET_DIR / 'anomal' / 'labels.txt'

assert (
    NORMAL_MEMORY_MAP.exists()
), f'Normal memory map does not exist: {NORMAL_MEMORY_MAP}'
assert (
    ANOMAL_MEMORY_MAP.exists()
), f'Anomal memory map does not exist: {ANOMAL_MEMORY_MAP}'
assert ANOMALIES_FILE.exists(), f'Anomalies file does not exist: {ANOMALIES_FILE}'

if MODEL_NAME == 'tae3d':
    assert (
        TIME_STEPS >= 16
    ), 'Number of time steps must be at least 16 for the 3D convolution model'

### LSTM

In [ ]:
# Simple test case for forward pass, also used as model signature in Mlflow.

if MODEL_NAME == 'tae':
    encoder = LSTMEncoder(n_time_steps=TIME_STEPS, bidirectional=True)
    decoder = LSTMDecoder(
        n_time_steps=TIME_STEPS,
        n_image_channels=1,
        image_size=IMAGE_SIZE,
        bidirectional=True,
    )

    # Test input tensor of size (batch_size, time_steps, channels, height, width)
    INPUT = torch.randn(BATCH_SIZE, TIME_STEPS, 1, IMAGE_SIZE, IMAGE_SIZE)

    # Forward pass through the encoder and decoder
    encoded = encoder(INPUT)
    decoded = decoder(encoded)

    # Check the shapes
    print(f'Input shape: {INPUT.shape}')
    print(f'Latent shape: {encoded.shape}')
    print(f'Decoded shape: {decoded.shape}')

    assert INPUT.shape == decoded.shape, 'Input and output shapes do not match!'

    print(summarize_model([encoder, decoder]))

### Conv3D

In [ ]:
# Simple test case for forward pass, also used as model signature in Mlflow.

if MODEL_NAME == 'tae3d':
    encoder = Conv3dEncoder()
    decoder = Conv3dDecoder()

    # Test input tensor of size (batch_size, channels, time_steps/depth, height, width)
    INPUT = torch.randn(BATCH_SIZE, 1, TIME_STEPS, IMAGE_SIZE, IMAGE_SIZE)

    # Forward pass through the encoder and decoder
    encoded = encoder(INPUT)
    decoded = decoder(encoded)

    # Check the shapes
    print(f'Input shape: {INPUT.shape}')
    print(f'Latent shape: {encoded.shape}')
    print(f'Decoded shape: {decoded.shape}')

    assert INPUT.shape == decoded.shape, 'Input and output shapes do not match!'

In [ ]:
# torchview.draw_graph(
#     Conv3dAutoencoder(),
#     input_size=(32, 1, 16, IMAGE_SIZE, IMAGE_SIZE),
#     graph_dir='TB',
#     depth=3,
#     roll=True,
#     expand_nested=True,
#     graph_name='Temporal Autoencoder',
#     save_graph=True,
#     filename='tae3d',
# ).visual_graph.render(format='svg')

## Loaders

In [ ]:
batch_size_dict = BatchSizeDict(
    {'train': BATCH_SIZE, 'valid': BATCH_SIZE, 'test': BATCH_SIZE}
)

train_val_dataset = TemporalAutoencoderDataset(
    memory_map_file=NORMAL_MEMORY_MAP,
    memory_map_image_shape=(IMAGE_SIZE, IMAGE_SIZE),
    window_size=TIME_STEPS,
    time_dim_index=0 if MODEL_NAME == 'tae' else 1,
)
test_dataset = TemporalAutoencoderDataset(
    memory_map_file=ANOMAL_MEMORY_MAP,
    memory_map_image_shape=(IMAGE_SIZE, IMAGE_SIZE),
    window_size=TIME_STEPS,
    time_dim_index=0 if MODEL_NAME == 'tae' else 1,
)

# Train validation split
train_size = int(TRAIN_SET_RATIO * len(train_val_dataset))
val_size = len(train_val_dataset) - train_size
train_dataset, valid_dataset = random_split(train_val_dataset, [train_size, val_size])

train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size_dict['train'],
    shuffle=False,  # YOU DON'T WANT TO SHUFFLE TEMPORAL DATA!
    num_workers=int(os.cpu_count()),  # type: ignore
    drop_last=True,
)

valid_dataloader = DataLoader(
    valid_dataset,
    batch_size=batch_size_dict['valid'],
    shuffle=False,
    num_workers=int(os.cpu_count()),  # type: ignore
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=1,  # batch_size_dict['test'],
    shuffle=False,
    num_workers=0,  # int(os.cpu_count()),  # type: ignore
)

## Training

In [ ]:
model = TemporalAutoencoderModel(
    encoder=encoder,
    decoder=decoder,
    batch_size_dict=batch_size_dict,
    learning_rate=LEARNING_RATE,
    loss_function=LOSS_FUNCTION,
    time_dim_index=1 if MODEL_NAME == 'tae' else 2,
    train_noise_std_input=TRAIN_NOISE_STD_INPUT,
    train_noise_std_latent=TRAIN_NOISE_STD_LATENT,
)

In [ ]:
csv_logger = CSVLogger(LOG_DIR, name=EXPERIMENT_NAME, version=VERSION)
early_stopping = EarlyStopping(
    monitor=MONITOR,
    mode='min',
    patience=PATIENCE,
)
model_checkpoint = ModelCheckpoint(
    dirpath=EXPERIMENT_DIR,
    filename='{epoch}-{val_loss:3f}',
    monitor=MONITOR,
    save_top_k=1,  # save only the best model
    mode='min',
)
progress_bar = TQDMProgressBar(refresh_rate=10)

In [ ]:
trainer = L.Trainer(
    accelerator='gpu',
    logger=csv_logger,
    callbacks=[model_checkpoint, early_stopping, progress_bar],
    max_epochs=MAX_EPOCHS,
    log_every_n_steps=1,  # log every batch
    # https://lightning.ai/docs/pytorch/stable/common/trainer.html#reproducibility
    deterministic=True,
)
torch.use_deterministic_algorithms(
    True, warn_only=True
)  # torch 2.5 does not have a deterministic implementation of `max_pool3d_with_indices_backward_cuda`

In [ ]:
# https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
torch.set_float32_matmul_precision('medium')

In [ ]:
trainer.fit(
    model,
    train_dataloaders=train_dataloader,
    val_dataloaders=valid_dataloader,
)

## Validation

In [ ]:
if False:
    # Load from MLflow
    model_name = 'pytorch-2024-11-02-073717-tae'
    model_version = 1
    model_uri = f'models:/{model_name}/{model_version}'
    model_ = mlflow.pytorch.load_model(model_uri)
else:
    model_checkpoint_path = list(EXPERIMENT_DIR.glob('*.ckpt'))[0]
    model_ = TemporalAutoencoderModel.load_from_checkpoint(
        model_checkpoint_path, encoder=encoder, decoder=decoder
    )

In [ ]:
trainer_ = L.Trainer(logger=False)  # no need to log anything for validation and testing

In [ ]:
valid_metrics = trainer_.validate(model_, dataloaders=valid_dataloader, verbose=True)[0]

In [ ]:
test_metrics = trainer_.test(model_, dataloaders=test_dataloader, verbose=True)[0]

In [ ]:
anomalies = Anomalies.from_file(ANOMALIES_FILE)
n_test_frames = len(list((ANOMALIES_FILE.parent / 'images').glob('*.jpg')))
y_true = anomalies.to_ground_truth(n_test_frames)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

errors = defaultdict(list)
model_.eval()
model_.to(device)
for i in tqdm(range(len(test_dataloader))):
    input_seq = test_dataloader.dataset[i]['image']
    with torch.no_grad():
        res = model_(input_seq.unsqueeze(0).to(device))[0]

    res = res.cpu()
    mse = (
        F.mse_loss(res, input_seq, reduction='none').mean(dim=1).squeeze().mean().item()
    )
    mae = (
        F.l1_loss(res, input_seq, reduction='none').mean(dim=1).squeeze().mean().item()
    )
    fro = torch.norm((res - input_seq).squeeze(), p='fro', dim=0).mean().item()

    errors['mse'].append(mse)
    errors['mae'].append(mae)
    errors['fro'].append(fro)

In [ ]:
def get_y_proba_from_errors(errors: list[float]) -> list[float]:
    """Normalize errors to [0, 1] range and get probabilistic `y_pred`."""
    min_err = min(errors)
    max_err = max(errors)
    errors_norm = [(x - min_err) / (max_err - min_err) for x in errors]
    y_proba = list(chain.from_iterable([[x] * TIME_STEPS for x in errors_norm]))
    return y_proba

In [ ]:
# Evaluate ROC AUC score for each metric, choose the best one.
best_metric = 'mse'
best_roc_auc_score = 0.0
for key, value in errors.items():
    y_proba = get_y_proba_from_errors(value)
    roc_auc = roc_auc_score(y_true, y_proba)
    if roc_auc > best_roc_auc_score:
        best_metric = key
        best_roc_auc_score = roc_auc

In [ ]:
roc_auc, optimal_threshold = plot_roc_chart(
    y_true,
    get_y_proba_from_errors(errors[best_metric]),
    save_path=EXPERIMENT_DIR / ROC_CHART_NAME,
    cbar_text=f'Thresholds ({best_metric.upper()})',
)

In [ ]:
plot_temporal_autoencoder_reconstruction(
    model_,
    test_dataloader,
    save_path=EXPERIMENT_DIR / PREDICTIONS_PNG_NAME,
    time_dim_index=0 if MODEL_NAME == 'tae' else 1,
    indices=[0]
    + [anomalies[0].middle() // TIME_STEPS]
    + [anomalies[2].middle() // TIME_STEPS],
)

In [ ]:
plot_learning_curves(
    EXPERIMENT_DIR / METRICS_CSV_NAME,
    save_path=EXPERIMENT_DIR / LEARNING_CURVES_PDF_NAME,
    metrics={
        'mse': 'Mean Squared Error',
        'fro': 'Frobenius Norm',
        'mae': 'Mean Absolute Error',
    },
    figsize=(22, 5),
)

In [ ]:
architecture_visualization = torchview.draw_graph(
    model_,
    input_size=INPUT.shape,
    graph_dir='TB',
    depth=3,
    roll=True,
    expand_nested=True,
    graph_name='Temporal Autoencoder',
    save_graph=False,
    filename=ARCHITECTURE_VISUALIZATION_NAME,
    directory=str(EXPERIMENT_DIR),
)
architecture_visualization.visual_graph.render(format='svg')
architecture_visualization.visual_graph.render(format='png')

## Logging

In [ ]:
def get_early_stopping_epoch() -> int | None:
    checkpoint = list(EXPERIMENT_DIR.glob('*.ckpt'))[0].stem
    pattern = r'epoch=(\d+)'
    match = re.search(pattern, checkpoint)
    if match:
        return int(match.group(1))
    else:
        return None

In [ ]:
def log_dict_to_mlflow(
    dictionary: Mapping[str, int | float], type: Literal['metric', 'param']
) -> None:
    for k, v in dictionary.items():
        if type == 'metric':
            mlflow.log_metric(k, v)
        elif type == 'param':
            mlflow.log_param(k, v)

In [ ]:
def get_submodule_param_count(model: L.LightningModule) -> dict[str, int]:
    """Get the number of parameters for each submodule in the model."""
    param_counts = {}
    for name, submodule in model.named_children():
        num_params = sum(p.numel() for p in submodule.parameters())
        if num_params > 0:
            param_counts[f'{name}_parameters'] = num_params
    param_counts['total_parameters'] = sum([x for x in param_counts.values()])
    return param_counts

In [ ]:
with mlflow.start_run(run_name=f'{EXPERIMENT_NAME}') as run:
    try:
        mlflow.set_tag('Branch', get_current_branch())
        mlflow.set_tag('Commit ID', get_commit_id())
        mlflow.set_tag('Dataset', DATASET_NAME)
    except Exception as e:
        print(e)

    mlflow.log_metric('roc_auc', roc_auc)
    mlflow.log_metric('optimal_threshold', optimal_threshold)
    log_dict_to_mlflow(valid_metrics, 'metric')
    log_dict_to_mlflow(test_metrics, 'metric')
    log_dict_to_mlflow(get_submodule_param_count(model_), 'param')

    mlflow.log_param('encoder', str(encoder))
    mlflow.log_param('decoder', str(decoder))
    mlflow.log_param('batch_size', BATCH_SIZE)
    mlflow.log_param('max_epochs', MAX_EPOCHS)
    mlflow.log_param('early_stopping', get_early_stopping_epoch())
    mlflow.log_param('monitor', MONITOR)
    mlflow.log_param('patience', PATIENCE)
    mlflow.log_param('image_size', IMAGE_SIZE)
    mlflow.log_param('learning_rate', LEARNING_RATE)
    mlflow.log_param('loss_function', LOSS_FUNCTION)
    mlflow.log_param('train_noise_std_input', TRAIN_NOISE_STD_INPUT)
    mlflow.log_param('train_noise_std_latent', TRAIN_NOISE_STD_LATENT)
    mlflow.log_param('seed', SEED)
    mlflow.log_param('time_steps', TIME_STEPS)
    mlflow.log_param('train_set_ratio', TRAIN_SET_RATIO)

    # CSV metrics, learning curves, predictions, notebook
    mlflow.log_artifact(str(EXPERIMENT_DIR / METRICS_CSV_NAME), MLFLOW_ARTIFACT_DIR)
    mlflow.log_artifact(
        str(EXPERIMENT_DIR / LEARNING_CURVES_PDF_NAME), MLFLOW_ARTIFACT_DIR
    )
    mlflow.log_artifact(str(EXPERIMENT_DIR / PREDICTIONS_PNG_NAME), MLFLOW_ARTIFACT_DIR)
    mlflow.log_artifact(str(EXPERIMENT_DIR / ROC_CHART_NAME), MLFLOW_ARTIFACT_DIR)
    mlflow.log_artifact(NOTEBOOK_NAME, MLFLOW_ARTIFACT_DIR)

    # Network visualization with `torchview`
    mlflow.log_artifact(
        str(EXPERIMENT_DIR / (ARCHITECTURE_VISUALIZATION_NAME + '.svg')),
        MLFLOW_ARTIFACT_DIR,
    )
    mlflow.log_artifact(
        str(EXPERIMENT_DIR / (ARCHITECTURE_VISUALIZATION_NAME + '.png')),
        MLFLOW_ARTIFACT_DIR,
    )

    # Models are versioned by default
    mlflow.pytorch.log_model(
        pytorch_model=model_,
        artifact_path='model',
        registered_model_name=f'pytorch-{EXPERIMENT_NAME}',
        signature=infer_signature(INPUT.numpy(), INPUT.numpy(), dict(training=False)),
    )